In [1]:
import pandas as pd
# import dummy classifier
from sklearn.dummy import DummyClassifier
# import train_test_split
from sklearn.model_selection import train_test_split

In [2]:
data = pd.read_csv('train.csv')

In [3]:
data.head()
data['age'] = round(data['age'] / 365)
target = data['cardio']

In [4]:
x_train, x_val, y_train, y_val = train_test_split(data.drop('cardio', axis=1), data['cardio'], test_size=0.2, random_state=0)

In [5]:
dummy_clf = DummyClassifier(strategy='most_frequent')
dummy_clf.fit(x_train, y_train)

dummy_clf.score(x_val, y_val)

0.4988392857142857

O KNN É um dos primeiros algoritmos aprendido no estudo de ML e é utilizado em problemas de classificação, ou sejam, tenta prever os resultados em uma saída discreta, em outras palavras, ele quer mapear variáveis ​​de entrada em categorias distintas.

o KNN tenta classificar cada amostra de um conjunto de dados avaliando sua distância em relação aos vizinhos mais próximos. Se os vizinhos mais próximos forem majoritariamente de uma classe, a amostra em questão será classificada nesta categoria.

In [6]:
#knn algorithm
from sklearn.neighbors import KNeighborsClassifier

knn_clf = KNeighborsClassifier()
knn_clf.fit(x_train, y_train)

knn_clf.score(x_val, y_val)

0.6325

In [7]:
#import min max scaler (normalizar os dados), one hot encoder (transformar em binários os dados categóricos), and column transformer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

# pode deixar dessa forma, se tiver mt ruim a gente muda
var_to_be_used = ['gluc', 'cholesterol', 'ap_hi', 'age', 'ap_lo', 'weight']
numerical_features = ['ap_hi', 'ap_lo', 'age', 'weight']
categorical_features = ['gluc', 'cholesterol']

# create column transformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', MinMaxScaler(), numerical_features),
        ('cat', OneHotEncoder(), categorical_features)
    ])

x_train = preprocessor.fit_transform(x_train)
x_val = preprocessor.transform(x_val)

In [8]:
import optuna
#import accuracy score
from sklearn.metrics import accuracy_score, precision_score
def objective(trial):
    """Define the objective function"""

    params = {
        'n_neighbors': trial.suggest_int('n_neighbors', 1, 100),
        'weights': trial.suggest_categorical('weights', ['uniform', 'distance']),
        'algorithm': trial.suggest_categorical('algorithm', ['auto', 'ball_tree', 'kd_tree', 'brute']),
        'leaf_size': trial.suggest_int('leaf_size', 1, 100),
        'p': trial.suggest_int('p', 1, 10),
    }

    # Fit the model
    optuna_model = KNeighborsClassifier(**params)
    optuna_model.fit(x_train, y_train)

    # Make predictions
    y_pred = optuna_model.predict(x_val)

    # Evaluate predictions
    accuracy = precision_score(y_val, y_pred)
    return accuracy
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100, n_jobs=8, timeout=600 * 3)

/home/bmmuc/.pyenv/versions/3.8.0/envs/estat/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2023-04-20 17:25:55,131] A new study created in memory with name: no-name-828fb98e-e97b-4368-b310-05b2909fd0c6
[I 2023-04-20 17:25:59,078] Trial 2 finished with value: 0.6864439324116743 and parameters: {'n_neighbors': 27, 'weights': 'uniform', 'algorithm': 'auto', 'leaf_size': 20, 'p': 10}. Best is trial 2 with value: 0.6864439324116743.
[I 2023-04-20 17:26:06,730] Trial 0 finished with value: 0.692573051948052 and parameters: {'n_neighbors': 37, 'weights': 'distance', 'algorithm': 'auto', 'leaf_size': 85, 'p': 3}. Best is trial 0 with value: 0.692573051948052.
[I 2023-04-20 17:26:11,529] Trial 3 finished with value: 0.684628804031789 and parameters: {'n_neighbors': 33, 'weights': 'uniform', 'algorithm': 'kd_

In [9]:
# train knn with the best params
trial = study.best_trial
params = trial.params
knn_clf = KNeighborsClassifier(**params)
# knn_clf.fit(x_train, y_train)

In [10]:
params

{'n_neighbors': 18,
 'weights': 'uniform',
 'algorithm': 'ball_tree',
 'leaf_size': 100,
 'p': 9}

In [18]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=5, shuffle=True, random_state=0)

# import cross_val_score
from sklearn.model_selection import cross_val_score

# evaluate the model using 10-fold cross-validation
scores = cross_val_score(knn_clf, x_train, y_train, cv=kf, scoring='precision', n_jobs=8)

#compute the mean of the scores and the standard deviation
print('precision: {:.2f} +/- {:.2f}'.format(scores.mean(), scores.std()))

precision: 0.70 +/- 0.01


In [12]:
knn_clf.fit(x_train, y_train)

KNeighborsClassifier(algorithm='ball_tree', leaf_size=100, n_neighbors=18, p=9)

In [13]:
data_test = pd.read_csv('test.csv')
test_target = data_test['cardio']
data_test['age'] = round(data_test['age'] / 365)

In [14]:
data_test['cardio'].shape

(14000,)

In [15]:
x_test = preprocessor.transform(data_test)
y_test = knn_clf.predict(x_test)

In [16]:
from sklearn.metrics import accuracy_score
precision_score(data_test['cardio'], y_test)

0.7055150884495317